In [1]:
import scipy
from astropy.io import fits
import h5py
import numpy as np
import drms
from glob import glob
from tqdm import tqdm

In [20]:
c = drms.Client(email = 'example@email.com', verbose=False)
series = 'hmi.sharp_cea_720s'
kwlist = ['T_REC','HARPNUM','USFLUX','LON_MIN','LON_MAX','LAT_MIN','LAT_MAX','NOAA_ARS','AREA',
          'CDELT1','CDELT2','RSUN_OBS','RSUN_REF','NAXIS1','NAXIS2',
          'CRPIX1','CRPIX2','CRVAL1','CRVAL2','CUNIT1','CUNIT2']
version = '1.0.0'

In [21]:
for sharp in tqdm(range(7050,9678)):
    sims = glob(f"D:\\MHS_solutions_v4\\sharp{sharp}.mat")

    if len(sims) > 0:
        simname = sims[0]
        sharpname = glob(f"D:\\sharp_data_v3\\*.{sharp}.*.Br.fits")[0]
        sharpstamp = sharpname.split('.')[3]
        sharpstamp_q = f'{sharpstamp[0:4]}.{sharpstamp[4:6]}.{sharpstamp[6:8]}_{sharpstamp[9:11]}:{sharpstamp[11:13]}:{sharpstamp[13:15]}'

        sharpimagename = glob(f"D:\\sharp_data_v3\\*.{sharp}.*.Br.fits")[0]
        sharp_hdul = fits.open(sharpimagename)
        
        k = c.query('%s[%d][%s]' % (series, sharp, sharpstamp_q), key=kwlist)

        try:
            mat = scipy.io.loadmat(simname)
        except NotImplementedError:
            mat = {}
            f = h5py.File(simname)
            for k,v in f.items():
                mat[k] = np.array(v)

        n = int(mat['n'])
        rns = np.concatenate(([[0]],np.array(mat['rns'])),0)
        nodes = np.array(mat['nodes'])
        Bs = np.concatenate((np.array(mat['Bff']),np.array(mat['Bns'])),1)
        Fs = np.concatenate((np.zeros((3*n,1)),np.array(mat['forcevec'])),1)
        
        for j in range(7):
            B = np.stack((Bs[0:n,j],Bs[n:2*n,j],Bs[2*n:3*n,j]),1)
            F = np.stack((Fs[0:n,j],Fs[n:2*n,j],Fs[2*n:3*n,j]),1)

            hdr = fits.Header()
            hdr['SIM_N'] = n
            if j == 0:
                hdr['SIM_L2'] = 'NULL'
            else:
                hdr['SIM_L2'] = rns[j,0]
            
            hdr['LEN_X'] = int(k.iloc[0].RSUN_REF / k.iloc[0].RSUN_OBS * k.iloc[0].CDELT1 * sharp_hdul[1].header['NAXIS1'])
            hdr['LEN_Y'] = int(k.iloc[0].RSUN_REF / k.iloc[0].RSUN_OBS * k.iloc[0].CDELT2 * sharp_hdul[1].header['NAXIS2'])
            hdr['LEN_Z'] = 2*max([hdr['LEN_X'],hdr['LEN_Y']])
            hdr['LEN_UNIT'] = 'meters'
            
            hdr['SHARPNUM'] = sharp
            hdr['ARNUM'] = k.iloc[0].NOAA_ARS
            hdr['TAI_REC'] = sharpstamp_q
            hdr['USFLUX'] = k.iloc[0].USFLUX
            hdr['AREA'] = k.iloc[0].AREA
            hdr['LON_MIN'] = k.iloc[0].LON_MIN
            hdr['LAT_MIN'] = k.iloc[0].LAT_MIN
            hdr['LON_MAX'] = k.iloc[0].LON_MAX
            hdr['LAT_MAX'] = k.iloc[0].LAT_MAX
            
            hdr['S_NAXIS1'] = sharp_hdul[1].header['NAXIS1']
            hdr['S_NAXIS2'] = sharp_hdul[1].header['NAXIS2']
            hdr['S_CRPIX1'] = k.iloc[0].CRPIX1
            hdr['S_CRPIX2'] = k.iloc[0].CRPIX2
            hdr['S_CRVAL1'] = k.iloc[0].CRVAL1
            hdr['S_CRVAL2'] = k.iloc[0].CRVAL2
            hdr['S_CUNIT1'] = k.iloc[0].CUNIT1
            hdr['S_CUNIT2'] = k.iloc[0].CUNIT2
            hdr['S_CDELT1'] = k.iloc[0].CDELT1
            hdr['S_CDELT2'] = k.iloc[0].CDELT2
            
            hdr['VERSION'] = version
            
            hdulist = fits.HDUList([
                fits.PrimaryHDU(header=hdr),
                fits.ImageHDU(Bs[0:n,j],name='BX'),
                fits.ImageHDU(Bs[n:2*n,j],name='BY'),
                fits.ImageHDU(Bs[2*n:3*n,j],name='BZ'),
                fits.ImageHDU(nodes[:,0],name='NODEX'),
                fits.ImageHDU(nodes[:,1],name='NODEY'),
                fits.ImageHDU(nodes[:,2],name='NODEZ'),
                fits.ImageHDU(Fs[0:n,j],name='FX'),
                fits.ImageHDU(Fs[n:2*n,j],name='FY'),
                fits.ImageHDU(Fs[2*n:3*n,j],name='FZ')
            ])

            hdulist.writeto(f'D:\\MHS_solutions_fits\\PARSE.simulation.{version}.sharp.{sharp}.realization.{j}.fits',overwrite=True,checksum=True)

100%|██████████████████████████████████████████████████████████████████████████████| 2628/2628 [22:31<00:00,  1.94it/s]


In [ ]:
# fits_image_filename = f"D:\sharp_data_v3\hmi.sharp_cea_720s.{sharp}.20170618_043600_TAI.Br.fits";
fits_image_filename = f'D:\\MHS_solutions_fits\\sharp{7050}.fits'
sharp_hdul = fits.open(fits_image_filename)
sharp_hdul[0].header

In [ ]:
for sharpfilename in glob(f"D:\\sharp_data_v3\\*.{sharp}.*.Br.fits"):
    sharptime = sharpfilename.split('.')[3]
    print(f'{sharptime[0:4]}.{sharptime[4:6]}.{sharptime[6:8]}_{sharptime[9:11]}:{sharptime[11:13]}:{sharptime[13:15]}')

In [ ]:
hdulist[0].header

In [ ]:
simname = glob(f"D:\\MHS_solutions_v4\\sharp{7000}.mat")
print(simname)

In [15]:
hdr

SIM_N   =                98968                                                  
SIM_L2  =   0.5930192490584691                                                  
LEN_X   =             11850875                                                  
LEN_Y   =              4267199                                                  
LEN_Z   =             23701750                                                  
LEN_UNIT= 'meters  '                                                            
SHARPNUM=                 7050                                                  
ARNUM   = '12663   '                                                            
TAI_REC = '2017.06.18_04:36:00'                                                 
USFLUX  =         6.688882E+21                                                  
AREA    =          2833.474365                                                  
LAT_MIN =             11.33781                                                  
LON_MIN =            16.6500

In [ ]:
k = c.query('%s[%d]' % (series, sharp), key=kwlist)

In [ ]:
fname = glob(f"D:\\sharp_data_v3\\*.{sharp}.*.Br.fits")[0]
hdul = fits.open(fname)

In [ ]:
type(hdul[1].data[0,0])

In [ ]:
hdul[0].header

In [ ]:
sharp = 7050
sims = glob(f"D:\\MHS_solutions_v4\\sharp{sharp}.mat")
simname = sims[0]
sharpname = glob(f"D:\\sharp_data_v3\\*.{sharp}.*.Br.fits")[0]
sharpstamp = sharpname.split('.')[3]
sharpstamp_q = f'{sharpstamp[0:4]}.{sharpstamp[4:6]}.{sharpstamp[6:8]}_{sharpstamp[9:11]}:{sharpstamp[11:13]}:{sharpstamp[13:15]}'

k = c.query('%s[%d][%s]' % (series, sharp, sharpstamp_q), key=kwlist)

In [ ]:
dir(k.iloc[0])

In [ ]:
k.iloc[0].NAXIS2

In [ ]:
int(k.iloc[0].RSUN_REF / k.iloc[0].RSUN_OBS * k.iloc[0].CDELT1 * hdul[1].header['NAXIS1'])

In [ ]:
k.iloc[0].CRPIX1

In [ ]:
k.iloc[0].CRVAL1

In [ ]:
k.iloc[0].RSUN_REF

In [ ]:
k.iloc[0].CDELT2

In [ ]:
max([k.iloc[0].RSUN_REF,k.iloc[0].CDELT2])